In [27]:
import os
import spark
import requests
from os.path import exists
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

In [30]:
spark = SparkSession.builder.appName('alcoholConsumption').getOrCreate()

In [3]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv'

In [4]:
response = requests.get(url)
if response.status_code == 200:
    spliter = url.split('/')
    filename = spliter[len(spliter) - 1]
    if not exists(filename):
        if open(filename, 'wb').write(response.content) == False:
            print('Download failed')
        else:
            print('Download succed')

Download succed


In [31]:
df = spark.read.csv(filename, header=True)

In [6]:
df.show()

+-----------------+-------------+---------------+-------------+----------------------------+---------+
|          country|beer_servings|spirit_servings|wine_servings|total_litres_of_pure_alcohol|continent|
+-----------------+-------------+---------------+-------------+----------------------------+---------+
|      Afghanistan|            0|              0|            0|                         0.0|       AS|
|          Albania|           89|            132|           54|                         4.9|       EU|
|          Algeria|           25|              0|           14|                         0.7|       AF|
|          Andorra|          245|            138|          312|                        12.4|       EU|
|           Angola|          217|             57|           45|                         5.9|       AF|
|Antigua & Barbuda|          102|            128|           45|                         4.9|       NA|
|        Argentina|          193|             25|          221|          

In [32]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- beer_servings: string (nullable = true)
 |-- spirit_servings: string (nullable = true)
 |-- wine_servings: string (nullable = true)
 |-- total_litres_of_pure_alcohol: string (nullable = true)
 |-- continent: string (nullable = true)



In [33]:
df = df.withColumn('beer_servings', col('beer_servings').cast(IntegerType()))

In [39]:
df = df.withColumn('wine_servings', col('wine_servings').cast(IntegerType()))

In [34]:
df.groupby('continent').sum('beer_servings').orderBy(col('sum(beer_servings)').desc()).show(1)

+---------+------------------+
|continent|sum(beer_servings)|
+---------+------------------+
|       EU|              8720|
+---------+------------------+
only showing top 1 row



In [35]:
df.show()

+-----------------+-------------+---------------+-------------+----------------------------+---------+
|          country|beer_servings|spirit_servings|wine_servings|total_litres_of_pure_alcohol|continent|
+-----------------+-------------+---------------+-------------+----------------------------+---------+
|      Afghanistan|            0|              0|            0|                         0.0|       AS|
|          Albania|           89|            132|           54|                         4.9|       EU|
|          Algeria|           25|              0|           14|                         0.7|       AF|
|          Andorra|          245|            138|          312|                        12.4|       EU|
|           Angola|          217|             57|           45|                         5.9|       AF|
|Antigua & Barbuda|          102|            128|           45|                         4.9|       NA|
|        Argentina|          193|             25|          221|          

In [37]:
gdf = df.groupby('continent')

In [38]:
sorted(gdf.agg({"*": "count"}).collect())

[Row(continent='AF', count(1)=53),
 Row(continent='AS', count(1)=44),
 Row(continent='EU', count(1)=45),
 Row(continent='NA', count(1)=23),
 Row(continent='OC', count(1)=16),
 Row(continent='SA', count(1)=12)]

In [48]:
gdf.agg(F.min('wine_servings'), F.max('wine_servings'), F.avg('wine_servings')).collect()

[Row(continent='AF', min(wine_servings)='0', max(wine_servings)='9', avg(wine_servings)=16.264150943396228),
 Row(continent='AS', min(wine_servings)='0', max(wine_servings)='9', avg(wine_servings)=9.068181818181818),
 Row(continent='EU', min(wine_servings)='0', max(wine_servings)='97', avg(wine_servings)=142.22222222222223),
 Row(continent='NA', min(wine_servings)='1', max(wine_servings)='9', avg(wine_servings)=24.52173913043478),
 Row(continent='OC', min(wine_servings)='0', max(wine_servings)='9', avg(wine_servings)=35.625),
 Row(continent='SA', min(wine_servings)='1', max(wine_servings)='8', avg(wine_servings)=62.416666666666664)]